# 3-3. Segmentation

In [1]:
!pip3 install --upgrade mxnet-cu101 > /dev/null
!pip3 install --upgrade gluoncv > /dev/null

tcmalloc: large alloc 1147494400 bytes == 0x38f7a000 @  0x7ff362590615 0x591e47 0x4cc179 0x4cc2db 0x50a1cc 0x50beb4 0x507be4 0x509900 0x50a2fd 0x50beb4 0x507be4 0x509900 0x50a2fd 0x50cc96 0x58e683 0x50c127 0x58e683 0x50c127 0x58e683 0x50c127 0x58e683 0x50c127 0x5095c8 0x50a2fd 0x50beb4 0x507be4 0x509900 0x50a2fd 0x50beb4 0x5095c8 0x50a2fd
ERROR: distributed 2021.1.1 has requirement dask>=2020.12.0, but you'll have dask 2.12.0 which is incompatible.
ERROR: botocore 1.19.59 has requirement urllib3<1.27,>=1.25.4; python_version != "3.4", but you'll have urllib3 1.24.3 which is incompatible.


In [2]:
!pip3 install --upgrade youtube-dl > /dev/null

In [3]:
def get_youtube_video(youtube_url):
  import youtube_dl

  ydl_opts = {  
    # 'format': 'mp4' 
    'format': 'best[height<=480][ext=mp4]' 
    } 
  with youtube_dl.YoutubeDL(ydl_opts) as ydl: 
      info_dict = ydl.extract_info(youtube_url, download=True)    
      filename = ydl.prepare_filename(info_dict)
  return filename

## video download

In [4]:
# country_road = get_youtube_video('https://www.youtube.com/watch?v=CObyXl__sEs')
# print(country_road)

In [5]:
city_road = get_youtube_video('https://www.youtube.com/watch?v=xNGo9shCeh8')
print(city_road)

[youtube] xNGo9shCeh8: Downloading webpage
[youtube] xNGo9shCeh8: Downloading js player bfb74eaf
[youtube] xNGo9shCeh8: Downloading js player bfb74eaf
[download] Destination: 40min City Drive in Frankfurt am Main, Germany  독일 프랑크푸르트 시내주행 주행 40분 연속 영상-xNGo9shCeh8.mp4
[download] 100% of 186.37MiB in 00:07
40min City Drive in Frankfurt am Main, Germany  독일 프랑크푸르트 시내주행 주행 40분 연속 영상-xNGo9shCeh8.mp4


In [6]:
# olympic = get_youtube_video('https://www.youtube.com/watch?v=agRTzCQXDQo')
# print(olympic)

In [7]:
incheon = get_youtube_video('https://www.youtube.com/watch?v=rSOgnrBsStU')
print(incheon)

[youtube] rSOgnrBsStU: Downloading webpage
[youtube] rSOgnrBsStU: Downloading MPD manifest
[download] Destination: 인천대로-경인고속도로 드라이브 South Korea Drive 001-rSOgnrBsStU.mp4
[download] 100% of 34.96MiB in 00:00
인천대로-경인고속도로 드라이브 South Korea Drive 001-rSOgnrBsStU.mp4


In [8]:
hanam = get_youtube_video('https://www.youtube.com/watch?v=CDF5NjRhHOA')
print(hanam)

[youtube] CDF5NjRhHOA: Downloading webpage
[youtube] CDF5NjRhHOA: Downloading MPD manifest
[download] Destination: Driving in Korea - Hanam city in Gyeonggi Province _ 경기도 하남 시내 드라이브-CDF5NjRhHOA.mp4
[download] 100% of 92.29MiB in 00:04
Driving in Korea - Hanam city in Gyeonggi Province _ 경기도 하남 시내 드라이브-CDF5NjRhHOA.mp4


## read

In [9]:
def read_video(filename, output_path, start_ms = 0, end_ms = None, step=1):
  from tqdm.notebook import tqdm
  import os 
  from gluoncv.utils.filesystem import try_import_cv2
  cv2 = try_import_cv2()
  # video_frames = [] 

  if not os.path.exists(output_path):
        os.makedirs(output_path)

  cap = cv2.VideoCapture(filename)

  cap.set(cv2.CAP_PROP_POS_MSEC, start_ms);

  width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
  height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
  frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
  fps = cap.get(cv2.CAP_PROP_FPS)
  total_ms = frame_count * fps * 1000
  print('Frame width:', width)
  print('Frame height:', height)
  print('Frame count:', frame_count)
  print('FPS:', fps)
  print('total sec:', int(total_ms/1000))

  capture_count = 0
  
  if end_ms != None:
    time_gap_sec = (end_ms - start_ms)/1000
    pbar_count = round(time_gap_sec*fps)
  else:
    pbar_count = round(frame_count)

  with tqdm(total=pbar_count) as pbar:
    while cap.isOpened():
        ret, img = cap.read()
        if not ret:
            cap.release()
            break 
        frame_no = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
        frame_msec = int(cap.get(cv2.CAP_PROP_POS_MSEC))

        # print('{}. msec:{}, endmsec:{}'.format(frame_no, frame_msec, end_ms)) 
        if end_ms != None and frame_msec >= end_ms:
            break

        # print('{}. msec:{}'.format(frame_no, frame_msec)) 

        # video_frames.append(img)
        cv2.imwrite(os.path.join(output_path, '%05d.jpg'%(frame_no)), img)
        capture_count += 1
        pbar.update(step)
        if step > 1:
            cap.set(cv2.CAP_PROP_POS_FRAMES, frame_no + (step-1));
  
  print('capture_count:', capture_count)

  return width, height, fps, frame_count

In [10]:
# !rm -rf city*

In [11]:
width, height, fps, frame_count = read_video(city_road, './city_road', 1000*60*28.5, 1000*60*29, 1)

Frame width: 640
Frame height: 360
Frame count: 76047
FPS: 30.0
total sec: 2281410



capture_count: 900


## 분석

In [12]:
import mxnet as mx
from mxnet import image
from mxnet.gluon.data.vision import transforms
import gluoncv 

In [13]:
psp = gluoncv.model_zoo.get_model('psp_resnet101_ade', pretrained=True)
# psp = gluoncv.model_zoo.get_model('psp_resnet101_citys', pretrained=True)

100121KB [00:02, 43676.56KB/s]                           


self.crop_size 480


100%|██████████| 266773/266773 [00:04<00:00, 63192.76KB/s]


In [14]:
def display_semantic_segmentation_mask(model, im_fname, height, color_pallete='ade20k'):
  import mxnet as mx
  from mxnet import image
  from mxnet.gluon.data.vision import transforms
  from matplotlib import pyplot as plt
  import gluoncv
  from gluoncv.utils.viz import get_color_pallete
  import matplotlib.image as mpimg
  from gluoncv.utils.filesystem import try_import_cv2
  cv2 = try_import_cv2()

  # using cpu
  ctx = mx.cpu(0) 
 
  # image loading(원본이미지를 그대로 불러옴.)
  from gluoncv import data 
  _, org_image = data.transforms.presets.yolo.load_test(im_fname, short=height)
  img = mx.nd.array(org_image) 

  # image pre transforming
  from gluoncv.data.transforms.presets.segmentation import test_transform
  img = test_transform(img, ctx)

  # inference
  output = model.predict(img)
  predict = mx.nd.squeeze(mx.nd.argmax(output, 1)).asnumpy()

  # display
  mask = get_color_pallete(predict, color_pallete)
  mask.save('output.png')
  mask_img = image.imread('output.png')
  
  #mask image와 원본이미지 합치기 투명도 0.8
  mask_img = gluoncv.utils.viz.cv_merge_two_images(org_image, mask_img, alpha=0.5)

  # cv BGR -> RGB
  mask_img = cv2.cvtColor(mask_img, cv2.COLOR_BGR2RGB)

  return mask_img

In [15]:
im_fname = gluoncv.utils.download('https://miro.medium.com/max/700/1*hO_md3xNtcncjAmGrTwxYA.jpeg')
mask_img = display_semantic_segmentation_mask(psp, im_fname , 480)

119KB [00:00, 2749.76KB/s]             

In [16]:
from matplotlib import pyplot as plt
plt.figure(figsize=(15,15)) 
ax = gluoncv.utils.viz.plot_image(mask_img ) 
plt.show()

<Figure size 1080x1080 with 0 Axes>

In [17]:
def get_semantic_video(detector,image_path, output_path, output_filename, width, height, fps, step=1): 
  from tqdm.notebook import tqdm
  import os 
  from gluoncv.utils.filesystem import try_import_cv2
  cv2 = try_import_cv2()

  if not os.path.exists(output_path):
    os.makedirs(output_path) 

  file_list = os.listdir(image_path)
  file_list.sort() 
  
  fourcc = cv2.VideoWriter_fourcc(*'FMP4')

  out = cv2.VideoWriter('{}/{}'.format(output_path, output_filename), fourcc, fps/step, (width, height))
 
  index = 0 # image id(for skip)
  for filename in tqdm(file_list):
    # step 값으로 이미지 skip
    if (index % step) != 0: 
      index += 1
      continue
    try:
      im_file = '{}/{}'.format(image_path,filename) 
      img = display_semantic_segmentation_mask(detector, im_file,   height) 
      frame = cv2.resize(img, dsize=(width, height), interpolation=cv2.INTER_AREA)
      out.write(frame) 
    except Exception as ex:
      pass 
    index += 1 # image id(for skip)
  out.release()

In [18]:
# !rm -rf city*

In [19]:
get_semantic_video(psp, './city_road', 'city_detect', 'city.mp4', 320, 240, 30.0, 5)

In [20]:
# !rm -rf hanam*

In [21]:
width, height, fps, frame_count = read_video(hanam, './hanam_road', 1000*60*9.5, 1000*60*10, 1)

Frame width: 640
Frame height: 360
Frame count: 35738
FPS: 29.970000648240262
total sec: 1071067



capture_count: 899


In [22]:
get_semantic_video(psp, './hanam_road', 'hanam_detect', 'hanam.mp4', 320, 240, 29.970000648240262, 5)

In [23]:
# !rm -rf incheon*
# !rm -rf city*
# !rm -rf hanam*

In [24]:
width, height, fps, frame_count = read_video(incheon, './incheon_road', 1000*60*6.5, 1000*60*7, 4)

Frame width: 640
Frame height: 360
Frame count: 12971
FPS: 30.0
total sec: 389130



capture_count: 225


In [25]:
get_semantic_video(psp, './incheon_road', 'incheon_detect', 'incheon.mp4', 320, 240, 30.0, 5)

# 실습

### 1. 아산 시내주행 드라이브 영상의 일부구간을 segmentation 영상으로 전환
- image 생성
- 이미지 처리하면서 detect 

In [26]:
asan = get_youtube_video('https://www.youtube.com/watch?v=t1YEq_2Un1Y')
print(asan)

[youtube] t1YEq_2Un1Y: Downloading webpage
[download] Destination: 아산 시내 드라이브 ASAN CITY DRIVING DOWNTOWN KOREA ROAD 4K 60P-t1YEq_2Un1Y.mp4
[download] 100% of 86.41MiB in 00:01
아산 시내 드라이브 ASAN CITY DRIVING DOWNTOWN KOREA ROAD 4K 60P-t1YEq_2Un1Y.mp4


In [27]:
width, height, fps, frame_count = read_video(asan, './asan_load', 1000*60*8, 1000*60*8.15, 4)

Frame width: 640
Frame height: 360
Frame count: 33295
FPS: 30.0
total sec: 998850



capture_count: 68


In [28]:
get_semantic_video(psp, './asan_load', 'asan_detect', 'asan.mp4', 320, 240, 30.0, 10)

### 2. 별에서 온 그대 하이라이트 영상의 일부구간을 segmentation 영상으로 전환
- image 생성
- 이미지 처리하면서 detect 


In [29]:
star = get_youtube_video('https://www.youtube.com/watch?v=PX3k11lAwc8')
print(star)

[youtube] PX3k11lAwc8: Downloading webpage
[download] Destination: [별에서온그대]  '내가 외계인을 좋아할 줄은 몰랐어..' 8분 하이라이트 _ 'My Love From the Star' Highlight _ SBS NOW-PX3k11lAwc8.mp4
[download] 100% of 28.74MiB in 00:00
[별에서온그대]  '내가 외계인을 좋아할 줄은 몰랐어..' 8분 하이라이트 _ 'My Love From the Star' Highlight _ SBS NOW-PX3k11lAwc8.mp4


In [32]:
width, height, fps, frame_count = read_video(star, './star_img', 1000*60*5, 1000*60*6.30, 4)

Frame width: 640
Frame height: 360
Frame count: 12697
FPS: 23.976065477677437
total sec: 304424



capture_count: 468


In [35]:
get_semantic_video(psp, './star_img', 'star_detect', 'star.mp4', 320, 240, 30.0, 10)